In [1]:
import numpy as np
from pathlib import Path
import mne
from tqdm.notebook import tqdm
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
#from mne.time_frequency import psd_welch
from acareeg.eegip import preprocessed_raw, process_events_resting_state, process_epochs, get_resting_state_epochs

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def get_onsets_resting_state(raw, age, dataset):
    if dataset == 'london':
        return get_onsets_london_resting_state(raw, age)
    
    if dataset == 'washington':
        return get_onsets_washington_resting_state(raw)

In [3]:
def get_onsets_washington_resting_state(raw):
    annot_sample = []
    annot_id = []  
    freq = raw.info["sfreq"]

    onsets = [a['onset'] for a in raw.annotations if a["description"] in ["Toys", "Socl"]]
    if len(onsets) == 0:
        return None

    if len(onsets) % 2 == 1:
        onsets.append(min(onsets[-1] + 60., raw.times[-1]))

    return list(zip(onsets[::2], onsets[1::2]))

In [4]:
def get_onsets_london_resting_state(raw, age):
    annot_sample = []
    annot_id = []  
    freq = raw.info["sfreq"]
    
    if age == 6:
        onsets = [a['onset'] for a in raw.annotations if a["description"] in ["Rst0", "Rst1"]]
        
        if len(onsets) == 0:
            return None
        if len(onsets) % 2 == 1:
            onsets.append(min(onsets + 40., raw.times[-1]))

        return list(zip(onsets[::2], onsets[1::2]))
    
    if age == 12:
        onsets = [annot["onset"] for annot in raw.annotations if annot["description"] == "Rst0"]
        
        if len(onsets) == 0:
            return None

        final_swir_swen_onset = [annot["onset"]
                                 for annot in raw.annotations
                                 if annot["description"] in ["SWEN", "SWIR"]][-1]
        durations = list(np.diff(onsets)) + [final_swir_swen_onset - onsets[-1]]

        # Truncate longer trials at 50s
        durations = [min(d, 50) for d in durations]

        # Reject any trial that last less than 10s
        durations, onsets = zip(*[(d, onset) for d, onset in zip(durations, onsets) if d >= 10])

        return list(zip(onsets, onsets+durations))

In [14]:
line_freqs = {"london": 50.0, "washington": 60.0}
pt = 45
age = 6
age_str = str(age)
dataset = 'washington'
interpolate = True
bids_root = 'C:/Users/dsrish/Documents/globus_eegip'
montage = mne.channels.make_standard_montage("GSN-HydroCel-129")

for apply_ica, file_name in zip([True, False],['ica', 'processed']):
    
    eeg_path = Path(bids_root) / dataset / "derivatives" / "lossless" / f"sub-s{pt:03}" / \
           f"ses-m{age:02}" / "eeg"
    eeg_path = list(eeg_path.glob("*_qcr.set"))
    eeg_path = eeg_path[0]
    raw = preprocessed_raw(eeg_path, line_freqs[dataset], montage=montage, reset_bads=True, 
                           apply_ica=apply_ica, interp_bad_ch=interpolate)
    
    
    raw_lst = []
    all_annots = mne.Annotations([],[],[])
    offset = 0
    for tmin, tmax in get_onsets_resting_state(raw, age, dataset):
        if offset == 0:
            offset = tmin
        else:
            offset+=tmin-previous_tmax
        previous_tmax = tmax
        raw_copy = raw.copy().crop(tmin=tmin, tmax=tmax)
        list_annot_values = [tuple(annot.values()) for annot in raw_copy.annotations] 
        onset, duration, description = list(zip(*list_annot_values))[:3]
        onset -= offset
        all_annots += mne.Annotations(onset,duration,description)
        raw_lst.append(raw_copy)
    
    raw_file = mne.concatenate_raws(raw_lst)
    
    raw_file.save(f'C:/Users/dsrish/Desktop/fieldtrip-20220208/s{str(0)}{pt}_m{str(age)}_rest_raw_{file_name}.fif',overwrite=True)


Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3301 samples (6.602 sec)



c:\users\dsrish\desktop\ds_msc_project\code\acareeg\acareeg\eegip.py:154: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(path, preload=True, verbose=verbose)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 129 out of 129 | elapsed:    3.9s finished
c:\users\dsrish\desktop\ds_msc_project\code\acareeg\acareeg\eegip.py:95: RuntimeWarning: Estimated head radius (10101.3 cm) is above the 99th percentile for adult head size. Check if the montage_units argument is correct (the default is "mm",

Applying ICA to Raw instance


c:\users\dsrish\desktop\ds_msc_project\code\acareeg\acareeg\eegip.py:147: RuntimeWarning: Estimated head radius (10101.3 cm) is above the 99th percentile for adult head size. Check if the montage_units argument is correct (the default is "mm", but your channel positions may be in different units).
  read_ica_eeglab(file_name).apply(raw, exclude=ind_comp_to_drop)


    Transforming to ICA space (105 components)
    Zeroing out 71 ICA components
    Projecting back using 105 PCA components
Writing C:\Users\dsrish\Desktop\fieldtrip-20220208\s045_m6_rest_raw_ica.fif
Closing C:\Users\dsrish\Desktop\fieldtrip-20220208\s045_m6_rest_raw_ica.fif
[done]


C:\Users\dsrish\AppData\Local\Temp/ipykernel_42712/1588881581.py:38: RuntimeWarning: This filename (C:\Users\dsrish\Desktop\fieldtrip-20220208\s045_m6_rest_raw_ica.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_file.save(f'C:/Users/dsrish/Desktop/fieldtrip-20220208/s{str(0)}{pt}_m{str(age)}_rest_raw_{file_name}.fif',overwrite=True)


Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3301 samples (6.602 sec)



c:\users\dsrish\desktop\ds_msc_project\code\acareeg\acareeg\eegip.py:154: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(path, preload=True, verbose=verbose)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 129 out of 129 | elapsed:    4.4s finished
c:\users\dsrish\desktop\ds_msc_project\code\acareeg\acareeg\eegip.py:95: RuntimeWarning: Estimated head radius (10101.3 cm) is above the 99th percentile for adult head size. Check if the montage_units argument is correct (the default is "mm",

Writing C:\Users\dsrish\Desktop\fieldtrip-20220208\s045_m6_rest_raw_processed.fif
Closing C:\Users\dsrish\Desktop\fieldtrip-20220208\s045_m6_rest_raw_processed.fif
[done]


C:\Users\dsrish\AppData\Local\Temp/ipykernel_42712/1588881581.py:38: RuntimeWarning: This filename (C:\Users\dsrish\Desktop\fieldtrip-20220208\s045_m6_rest_raw_processed.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_file.save(f'C:/Users/dsrish/Desktop/fieldtrip-20220208/s{str(0)}{pt}_m{str(age)}_rest_raw_{file_name}.fif',overwrite=True)
